In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

In [4]:
train = add_features(train)
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,area,u_in_cumsum,u_in_lag1,u_out_lag1,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083334,0,5.837492,0.000000,0.083334,0.000000,0.0,...,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033652,18.383041,0,5.907794,0.618632,18.466375,0.083334,0.0,...,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067514,22.509278,0,7.876254,2.138333,40.975653,18.383041,0.0,...,1,0,0,1,0,0,0,0,0,0
3,4,1,0.101542,22.808822,0,11.742872,4.454391,63.784476,22.509278,0.0,...,1,0,0,1,0,0,0,0,0,0
4,5,1,0.135756,25.355850,0,12.234987,7.896588,89.140326,22.808822,0.0,...,1,0,0,1,0,0,0,0,0,0


In [5]:
train.drop(['id','breath_id'], axis = 1, inplace = True)
train.head()

,time_step,u_in,u_out,pressure,area,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,0.000000,0.083334,0,5.837492,0.000000,0.083334,0.000000,0.0,18.383041,0.0,...,1,0,0,1,0,0,0,0,0,0
1,0.033652,18.383041,0,5.907794,0.618632,18.466375,0.083334,0.0,22.509278,0.0,...,1,0,0,1,0,0,0,0,0,0
2,0.067514,22.509278,0,7.876254,2.138333,40.975653,18.383041,0.0,22.808822,0.0,...,1,0,0,1,0,0,0,0,0,0
3,0.101542,22.808822,0,11.742872,4.454391,63.784476,22.509278,0.0,25.355850,0.0,...,1,0,0,1,0,0,0,0,0,0
4,0.135756,25.355850,0,12.234987,7.896588,89.140326,22.808822,0.0,27.259866,0.0,...,1,0,0,1,0,0,0,0,0,0


In [12]:
# normalise
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# RS = RobustScaler()
# train = RS.fit_transform(train)
# train.shape

In [10]:
features = [x for x in train.columns.values if x != 'pressure']
X = train[features]
y = train.pressure

train_X,val_X,train_y,val_y=train_test_split(X,y,random_state=1)

In [13]:
model = RandomForestRegressor(max_depth = 10, n_jobs = -1, random_state = 1)

In [14]:
model.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=1)

In [15]:
predictions = model.predict(val_X)
val_mae = mean_absolute_error(predictions, val_y)
print(f"Validation MAE for Random Forest:{val_mae}")

Validation MAE for Random Forest:0.9655574703185837


In [37]:
# test = add_features(test)
# test_X = test.drop(['id','breath_id'], axis = 1, inplace = True)
test = pd.read_csv("test.csv")
test = add_features(test)
test.head()

,id,breath_id,time_step,u_in,u_out,area,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,7.515046,...,0,0,0,0,0,0,0,0,1,0
1,2,0,0.031904,7.515046,0,0.239758,7.515046,0.000000,0.0,14.651675,...,0,0,0,0,0,0,0,0,1,0
2,3,0,0.063827,14.651675,0,1.174935,22.166721,7.515046,0.0,21.230610,...,0,0,0,0,0,0,0,0,1,0
3,4,0,0.095751,21.230610,0,3.207788,43.397331,14.651675,0.0,26.320956,...,0,0,0,0,0,0,0,0,1,0
4,5,0,0.127644,26.320956,0,6.567489,69.718287,21.230610,0.0,30.486938,...,0,0,0,0,0,0,0,0,1,0


In [38]:
test_without_id = test.drop(['id','breath_id'], axis = 1)
test_without_id.head()

,time_step,u_in,u_out,area,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,7.515046,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,0
1,0.031904,7.515046,0,0.239758,7.515046,0.000000,0.0,14.651675,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,0
2,0.063827,14.651675,0,1.174935,22.166721,7.515046,0.0,21.230610,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,0
3,0.095751,21.230610,0,3.207788,43.397331,14.651675,0.0,26.320956,0.0,7.515046,...,0,0,0,0,0,0,0,0,1,0
4,0.127644,26.320956,0,6.567489,69.718287,21.230610,0.0,30.486938,0.0,14.651675,...,0,0,0,0,0,0,0,0,1,0


In [39]:
test_preds = model.predict(test_without_id)
print(test_preds[0:20])

[ 6.15870677  5.85316111  7.37916886  8.2801608   9.56082707 10.15228963
 10.43949302 14.1996017  15.12288085 15.12288085 17.4111916  17.4111916
 17.4111916  18.32718559 20.90384133 20.90384133 20.90384133 20.90384133
 20.90384133 20.90384133]


In [40]:
output = pd.DataFrame({'id': test.id,'pressure': test_preds})
output.to_csv('submission.csv', index = False)
print('end')

end
